# Analyze and plot repetition code memory experiment data

In [1]:
from typing import Dict, List

import collections
import itertools
import functools

import matplotlib.pyplot as plt; plt.rcParams.update({"font.family": "serif"})
import numpy as np
import pickle

import cirq
import qiskit

## Helper functions

In [15]:
def get_counts_for(qubit: int, counts: collections.Counter[str, int]) -> collections.Counter[int, int]:
    qubit_counts = collections.Counter()
    for key, value in counts.items():
        k = list(key)[qubit]
        qubit_counts[int(k)] += value
    return qubit_counts


def generate_stabilizer_elements(generators: List[cirq.PauliString]) -> List[cirq.PauliString]:
    elements = []
    for string in itertools.chain.from_iterable(itertools.combinations(generators, r) for r in range(len(generators) + 1)):
        elements.append(
            functools.reduce(lambda a, b: a * b, string, cirq.PauliString())
        )
    return elements


def compute_expectation(
    pauli: cirq.PauliString,
    counts: Dict[str, int],
) -> float:
    if pauli is cirq.PauliString():
        return 1.0

    expectation = 0.0

    indices = [q.x for q in pauli.qubits]
    for key, value in counts.items():
        key = list(map(int, list(key)))
        expectation += (-1) ** sum([key[i] for i in indices]) * value

    return expectation / sum(counts.values())

## Load data

In [2]:
# Set data directory (job id).
job_id = "ctwgnt7seb10008x520g"

In [11]:
with open(f"{job_id}/{job_id}.pkl", "rb") as f:
    result = qiskit.result.Result.from_dict(pickle.load(f))


nqubits = int(np.loadtxt(f"{job_id}/nqubits.txt"))
depths = [int(d) for d in np.loadtxt(f"{job_id}/depths.txt")]
nvals = [int(n) for n in np.loadtxt(f"{job_id}/nvals.txt")]
physical_qubits = [int(q) for q in np.loadtxt(f"{job_id}/physical_qubits.txt")]
nshots = int(np.loadtxt(f"{job_id}/nshots.txt"))

## Visualize physical results

In [12]:
expectations = {q: [] for q in range(nqubits)}

for counts in result.get_counts():  # Loop over circuit depth.
    for qubit in expectations.keys():
        qubit_counts = get_counts_for(qubit, counts)
        expectations[qubit].append(
            (qubit_counts.get(0) - qubit_counts.get(1)) / nshots
        )

expectations = {q: np.array(v) for q, v in expectations.items()}

In [ ]:
for qubit, expectation in expectations.items():
    plt.plot(depths, expectation, "--o", label=f"Qubit {physical_qubits[qubit]}", alpha=0.75, mec="black")

plt.title(result.header.backend_name + "\njob id: " + job_id)

plt.xlabel("Depth")
plt.ylabel(r"$\langle Z \rangle$")
plt.legend(ncol=2);

In [ ]:
all_expectations = np.array([v for _, v in expectations.items()])

plt.errorbar(
    x=depths,
    y=np.average(all_expectations, axis=0),
    yerr=np.std(all_expectations, axis=0, ddof=1),
    capsize=8,
    lw=2,
)

plt.title(result.header.backend_name + "\njob id: " + job_id)

plt.xlabel("Depth")
plt.ylabel(r"Average $\langle Z \rangle$");


## Post-processing

In [16]:
# # Post-process by applying the codespace projector directly. Note that for this experiment this equivalent to the (much) faster post-processing by post-selection below.
# all_lst_values = {}

# for n in nvals:
#     print(f"Post-processing status: n = {n}")
#     q = cirq.LineQubit.range(n)
#     observable = cirq.Z.on(q[0])
#     generators = [
#         cirq.Z.on(q[i]) * cirq.Z.on(q[i + 1]) for i in range(len(q) - 1)
#     ]
#     elements = generate_stabilizer_elements(generators)
#     paulis = [observable * p for p in elements]
#     lst_values = []

#     for counts in result.get_counts():
#         denominator = sum(
#             compute_expectation(e, counts) for e in elements
#         ) / 4.0
#         numerator = sum(
#             compute_expectation(p, counts) for p in paulis
#         ) / 4.0
#         lst_values.append(numerator / denominator)

#     print(lst_values)
#     all_lst_values[n] = lst_values

In [ ]:
# Post-process by post-selection.
all_lst_values = {}

for n in nvals:
    print(f"Post-processing status: n = {n}")
    post_selection_values = [] = []

    for counts in result.get_counts():
        subselected = collections.Counter()
        for k, v in counts.items():
            subselected[k[:n]] += v

        num_zero = subselected.get("0" * n, 0)
        num_one   = subselected.get("1" * n, 0)

        total = num_zero + num_one
        eval = (num_zero - num_one) / total
        post_selection_values.append(eval)

    all_lst_values[n] = post_selection_values

## Plotting

In [ ]:
# Plot average physical expectation values.
plt.errorbar(
    x=depths,
    y=np.average(all_expectations, axis=0),
    yerr=np.std(all_expectations, axis=0, ddof=1),
    capsize=8,
    lw=2,
    label="Physical",
)

# Plot LST expectation values.
for n, lst_values in all_lst_values.items():
    plt.plot(depths, lst_values, "--s", markersize=10, label=f"$n = {n}$", alpha=0.75, mec="black")

# Format.
# plt.title(result.header.backend_name + "\njob id: " + job_id)
plt.xlabel("$d$")
plt.ylabel(r"$\langle Z \rangle$")
plt.legend();
plt.savefig(f"{job_id}_d.pdf")

In [ ]:
lst_vals = np.array([v for _, v in all_lst_values.items()]).T
for i, vals in enumerate(lst_vals):
    plt.plot(nvals, vals, "--o", label=f"$d = {depths[i]}$", markersize=8, mec="black", alpha=0.75)


plt.xticks(np.arange(min(nvals), max(nvals) + 1, 1))
plt.xlabel("$n$")
plt.ylabel(r"$\langle Z \rangle$")
plt.legend();
plt.savefig(f"{job_id}_n.pdf")